In [1]:
import requests
import random
import base64
from Crypto.Cipher import AES
import json
import binascii
import os
import requests
import re
import pandas as pd
class User_api():
    # 设置从JS文件提取的RSA的模数、协商的AES对称密钥、RSA的公钥等重要信息
    def __init__(self):
        self.modulus = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
        self.nonce = '0CoJUm6Qyw8W8jud'
        self.pubKey = '010001'
        self.url = "https://music.163.com/weapi/cloudsearch/get/web?csrf_token="
        self.HEADER = {}
        self.setHeader()
        self.secKey = self.getRandom()

    # 生成16字节即256位的随机数
    def getRandom(self):
        string = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        res = ""
        for i in range(16):
            res += string[int(random.random()*62)]
        return res

    # AES加密，用seckey对text加密
    def aesEncrypt(self, text, secKey):
        pad = 16 - len(text) % 16
        text = text + pad * chr(pad)
        encryptor = AES.new(secKey.encode('utf-8'), 2, '0102030405060708'.encode('utf-8'))
        ciphertext = encryptor.encrypt(text.encode('utf-8'))
        ciphertext = base64.b64encode(ciphertext).decode("utf-8")
        return ciphertext

    # rsa加密
    def rsaEncrypt(self, text, pubKey, modulus):
        text = text[::-1]
        a = int(binascii.hexlify(str.encode(text)), 16)
        b = int(pubKey, 16)
        c = int(modulus, 16)
        rs = self.quickpow(a, b, c)
        return format(rs, 'x').zfill(256)

    # 设置请求头
    def setHeader(self):
        self.HEADER = {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip,deflate,sdch',
            'Accept-Language': 'zh-CN,zh;q=0.8,gl;q=0.6,zh-TW;q=0.4',
            'Connection': 'keep-alive',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Host': 'music.163.com',
            'Referer': 'https://music.163.com/search/',
            'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.152 Safari/537.36'
        }

    # 设置相应的请求参数，从而搜索列表
    # 总体的密码加密步骤为：
    # 首先用nonce对text加密生成密文1
    # 然后用随机数seckey加密密文1生成密文2
    # 随后，用公钥加密seckey生成密文3
    # 其中，密文2作为请求参数中的params，密文3作为encSeckey字段
    # 这样，接收方可以通过私钥解密密文3获得seckey(随机数)
    # 然后用seckey解密密文2获得密文1
    # 最终用统一协商的密钥nonce解密密文1最终获得text
    def search(self,s):
        text = {
            "hlpretag": "<span class=\"s-fc7\">",
            "hlposttag": "</span>",
            "s": s,
            "type": "1002",
            "offset": "0",
            "total": "true",
            "limit": "100",
            "csrf_token": ""
        }
        text = json.dumps(text)
        params = self.aesEncrypt(self.aesEncrypt(text,self.nonce),self.secKey)
        encSecKey = self.rsaEncrypt(self.secKey,self.pubKey,self.modulus)
        data = {
            'params': params,
            'encSecKey': encSecKey
        }
        result = requests.post(url=self.url,
                                data=data,
                                headers = self.HEADER).json()
        return result
        # 快速模幂运算，求 x^y mod mo
    def quickpow(self, x, y, mo):
        res = 1
        while y:
            if y & 1:
                res = res * x % mo
            y = y // 2
            x = x * x % mo
        return res

In [2]:
def look_for_id(name):
    '''
    :param name:
    name:输入你的用户名,应当为str类型
    :return:
    1.返回你真正的id。
    2.如果没有查找到，则返回第一个id。
    '''
    try:
        a=User_api().search(name)['result']['userprofiles']
        nickname=[]
        nickid=[]
        for i in range(len(a)):
            nickname.append(a[i]['nickname'])
            nickid.append(a[i]['userId'])
        suoyin=nickname.index(name)
        result=nickid[suoyin]
        return result
    except:
        result=User_api().search(name)['result']['userprofiles'][0]['userId']
        return result

In [3]:
def get_data(uid):
    '''得到指定用户的基本json信息'''
    '''进入登陆状态的session'''
    login_url='http://localhost:3000/login/cellphone?phone=13408462303&password=shiyan123456789'
    s=requests.Session()
    responsesdata1=s.get(url=login_url)
    uid=uid

    recharge_url='http://localhost:3000/user/playlist?uid='+str(uid)
    responsesdata2=s.get(url=recharge_url)
    result=json.loads(responsesdata2.text)
    if(result['code']==200):
        with open(str('用户歌单/')+str(uid)+".json", "w",encoding='utf-8') as f:
            f.write(json.dumps(result, ensure_ascii=False, indent=4, separators=(',', ':')))
    if(result['code']!=200):
        return 0
    with open('用户歌单/'+str(uid)+'.json','r',encoding='utf8')as fp:
        t_data = json.load(fp)
    temp_list_ids=[]
    if(len(t_data['playlist'])==0):
        return 0

In [4]:
def get_user_songlist_id(uid):
    '''解读前面爬下来的json数据，获得用户歌单id
    返回的是歌单id的列表
    '''
    get_data(uid)
    with open('用户歌单/'+str(uid)+'.json','r',encoding='utf8')as fp:
        json_data = json.load(fp)
    temp_list_ids=[]
    if(len(json_data['playlist'])==0):
        return 0
    for i in range(len(json_data['playlist'])):
        temp_list_ids.append(json_data['playlist'][i]['id'])
    return temp_list_ids

In [5]:
def get_name(list_data):
    headers={
    'Referer':'https://music.163.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.89 Safari/537.36',
    'Connection': 'close',
    }
    list_id = list_data

    list_url = 'https://music.163.com/song?id=%s' % list_id
    res = requests.get(list_url, headers=headers)


    r1 = res.text
    contentlist=[]
    artist_pattern = r"""content="https://music.163.com/artist\?id=(.*?)"/>"""
    list_artist = re.findall(artist_pattern,r1)
    song_title = r"""<em class="f-ff2">(.*?)</em>"""
    list_song = re.findall(song_title,r1)
    contentlist.append(list_song[0])
    contentlist.append(list_artist[0])
    return contentlist

In [6]:
def get_users_musiclist(name):
    '''得到用户的歌单频率表'''
    name=name
    temp_id=look_for_id(name)
    get_data(temp_id)
    a=get_user_songlist_id(temp_id)
    data=pd.DataFrame({'歌曲id':[],'歌曲频率':[],'歌曲名字':[],'歌手id':[]})
    for count in range(len(a)):
        temp_list=a[count]
        login_url='http://localhost:3000/login/cellphone?phone=13408462303&password=shiyan123456789'
        s=requests.Session()
        responsesdata1=s.get(url=login_url)
        recharge_url='http://localhost:3000/playlist/detail?id='+str(temp_list)
        responsesdata2=s.get(url=recharge_url)
        result=json.loads(responsesdata2.text)
        result=result['playlist']['trackIds']
        songs_id=[]
        songs_listen_number=[]
        song_name=[]
        singer_id=[]
        for i in range(len(result)):
            songs_id.append(str(result[i]['id']))
            songs_listen_number.append(result[i]['v'])
            temp_func=get_name(str(result[i]['id']))
            print(temp_func)
            song_name.append(temp_func[0])
            singer_id.append(temp_func[1])
        temp_data=pd.DataFrame({'歌曲id':songs_id,'歌曲频率':songs_listen_number,
                                '歌曲名字':song_name,'歌手id':singer_id})
        data=pd.concat([data,temp_data])
    return data

In [7]:
name='daixxzz'
id=look_for_id(name)
id

1483982859

In [8]:
data=get_users_musiclist(name)
data.to_excel('代.xlsx',index=False,encoding='ANSI')
data.to_csv('代.csv',index=False)

['夜里无星', '12676697']
['99', '13058502']
['喜新恋旧', '784257']
['慢慢喜欢你', '8926']
['不想喝酒！', '12292356']
['梦醒时分', '12383009']
['夜车', '10750']
['Almost\xa0Home', '1079074']
['讨喜', '12050057']
['Someday or One Day', '954154']
['My Man', '784257']
['与海无关', '12676697']
['Midsummer Madness', '15022053']
['黑马王子', '1132392']
['花花公子 (feat. step.jad)', '1132392']
['不会说谎的恰恰 (Live)', '1132392']
['R&amp;B All Night (Masiwei Remix)', '1132392']
['Tonight', '31668975']
['너의 하루 (바른연애 길잡이 X 로꼬)', '124303']
['180度', '9272']
['Devastated', '31215569']
['不会说谎的眼睛 (Feat. DP龙猪)', '1132392']
['一无所有', '12200029']
['拖延症', '28420071']
['Summer Go Loco', '124303']
['最后的831 (万人合唱版)', '11109']
['一颗苹果', '13193']
['MOVIE', '28102242']
['克卜勒', '9272']
['拥抱', '13193']
['Y U Gotta B Like That', '33590146']
['一天', '33885397']
['回不去的夏天', '12383009']
['欢迎光临', '784257']
['想去海边', '12383009']
['无法坠入爱河 (Punk Version)', '1151001']
['百变酒精', '865007']
['花,太阳,彩虹,你', '6309']
['fine!', '12108736']
['Intentions', '35531']
['Dawn of us', '